<div align='left' style="width:29%;overflow:hidden;">
<a href='http://inria.fr'>
<img src='https://github.com/lmarti/jupyter_custom/raw/master/imgs/inr_logo_rouge.png' alt='Inria logo' title='Inria logo'/>
</a>
</div>

# Representations, LDA and topics in CORD-19

> Here we calculate representations of the papers based on their text content. Then, from these representations, a modeling of topics will be carried out using the LDA method. Finally, the most relevant papers for each topic will be determined using the PageRank scores of each paper.

In [ ]:
# default_exp lda

In [ ]:
!pip install -q -r requirements.txt

In [ ]:
# export
from risotto.references import load_papers_from_metadata_file, build_papers_reference_graph, paper_as_markdown
from fastprogress.fastprogress import progress_bar

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from pathlib import Path

import scispacy
import en_core_sci_sm
import networkx as nx
from collections import defaultdict
import numpy as np

Loading paper dataset and re-generating the graph of papers and the corresponding PageRank.

In [ ]:
cord19_dataset_folder = "./datasets/CORD-19-research-challenge"

In [ ]:
papers, _ = load_papers_from_metadata_file(cord19_dataset_folder)

In [ ]:
G = build_papers_reference_graph(papers)

In [ ]:
pageranks = nx.pagerank(G)

## Paper representations

In order to build a representation for each paper, the following libraries will be used:

- spaCy: https://spacy.io/
- scispaCy: https://allenai.github.io/scispacy/

The language model named`en_core_sci_sm` will be used, which has been trained with a corpus of biomedical text with a vocabulary of more than 100.000 words.
In case of needing a model with a larger vocabulary, there are some others available.

Loading the biomedical language pipeline.

In [ ]:
# export
nlp = en_core_sci_sm.load()

In [ ]:
# Select a paper to showcase spacy's features
sample_paper = list(pageranks.keys())[0]
sample_text = "\n".join([ paragraph["text"] for paragraph in sample_paper._file_contents["body_text"]])
sample_text

doc = nlp(sample_text, disable=["tagger", "parser", "ner"])
doc[17].lemma_

'be'

The document tokenized by the `spacy` pipeline is displayed.
An interesting thing about using `spacy` with the pretrained language model is that it automatically computes document and token representations vectors.
It's a pending task to find out which language model architecture it's used to compute those vectors.

A relevant aspect that influences downstream tasks is the number of out-of-vocabulary (OOV) tokens.
The following cell makes a quick inspection over a sample paper counting the number of OOV tokens.
A continuación, se realizará una iteración sobre los tokens para detectarlos.

In [ ]:
num_oov = 0
for token in doc:
    if token.is_oov and token.string != "\n":
        if token.string.endswith("virus"):
            print(token, "not found")
        num_oov += 1
    else:
        if token.string.endswith("virus"):
            print(token, "found")

virus found
virus found
virus found


In [ ]:
print(f'Number of out of vocabulary tokens: {num_oov} ({100 * num_oov / len(doc)}%).')

Number of out of vocabulary tokens: 336 (4.801371820520148%).


Note that relevant tokens, such as *coronavirus* are included in the language model vocabulary.

Testing the mechanisms used to remove stopwords, punctuation, spaces, and extract the token's lemma.

In [ ]:
tokens = {token for token in doc}

In [ ]:
no_stop_word_tokens = {token for token in doc if not (token.is_stop or token.is_punct or token.is_space)}

In [ ]:
len(tokens), len(no_stop_word_tokens)

(6998, 3544)

## Latent Dirichlet Allocation (LDA)

The following cells will perform topic modelling experiments using the LDA technique.
The `scikit-learn` implementation of this model will be used.

First, let's process all documents texts.

In [ ]:
# export
def process_papers_file_contents(papers):
    texts = []
    nlp = en_core_sci_sm.load()
    for paper in progress_bar(papers):
        text = " \n ".join([ paragraph["text"] for paragraph in paper._file_contents["body_text"]])
        """
        NB.: for development speed purposes, the only document's attributes
        considered for the topic modelling were the title and the abstract.
        Should the text be included in other experiments, the following line
        should be modified to include `{paper.text}`.
        """
        texts.append(f"{paper.title} \n {paper.abstract}")
    return texts

In [ ]:
docs = process_papers_file_contents(list(pageranks.keys()))

Peeking at the top 5 papers.

In [ ]:
print('\n=====\n'.join(docs[:5]))

Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synthesis 
 Nidovirus subgenomic mRNAs contain a leader sequence derived from the 5′ end of the genome fused to different sequences (‘bodies’) derived from the 3′ end. Their generation involves a unique mechanism of discontinuous subgenomic RNA synthesis that resembles copy-choice RNA recombination. During this process, the nascent RNA strand is transferred from one site in the template to another, during either plus or minus strand synthesis, to yield subgenomic RNA molecules. Central to this process are transcription-regulating sequences (TRSs), which are present at both template sites and ensure the fidelity of strand transfer. Here we present results of a comprehensive co-variation mutagenesis study of equine arteritis virus TRSs, demonstrating that discontinuous RNA synthesis depends not only on base pairing between sense leader TRS and antisense body TRS, but also on the primary sequence o

Vectors storing the token occurrence count will be used as document representations.
`tf-idf` vectors are purposefully not used because the document frequency normalization is already carried out by the LDA technique.

In [ ]:
# export
def tokenizer(sentence):
    tokens = []
    for token in nlp(sentence, disable=["tagger", "parser", "ner"]):
        # Se descartan números, stopwords, puntuación, espacio y tokens de largo 1
        if not (token.like_num or token.is_stop or token.is_punct
                or token.is_space or len(token) == 1):
            tokens.append(token.lemma_)
    return tokens

In [ ]:
count_vectorizer = CountVectorizer(tokenizer=tokenizer, lowercase=True)

In [ ]:
vectorized_docs = count_vectorizer.fit_transform(docs)

A sparse matrix is built rows one for each document, and columns: one for each token.

In [ ]:
vectorized_docs.shape

(44648, 164664)

In [ ]:
len(count_vectorizer.vocabulary_)

164664

In [ ]:
lda = LatentDirichletAllocation(n_components=10, verbose=2, n_jobs=-1)
lda = lda.fit(vectorized_docs)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  1.4min remaining:  1.4min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  1.6min finished


iteration: 1 of max_iter: 10


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  1.6min remaining:  1.6min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  1.7min finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 2 of max_iter: 10


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   57.9s remaining:   57.9s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  1.0min finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   35.8s remaining:   35.8s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   37.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 9 of max_iter: 10


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   37.0s remaining:   37.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   40.4s finished


iteration: 10 of max_iter: 10


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   31.7s remaining:   31.7s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   33.3s finished


In [ ]:
lda

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=-1,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=2)

The execution of the following cells will display the most relevant tokens for each identified topic.

In [ ]:
# export
def topic_descriptors(topic_model, vectorizer, num_words):
    res = {}
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(topic_model.components_):
        res[topic_idx] =[feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]
    return res

In [ ]:
descriptors = topic_descriptors(lda, count_vectorizer, 10)

In [ ]:
for topic_id in descriptors:
    print(f'Topic {topic_id}:', ', '.join(descriptors[topic_id]))

Topic 0: cell, infection, response, expression, immune, mouse, disease, viral, gene, lung
Topic 1: activity, antiviral, drug, inhibitor, compound, effect, treatment, protease, study, cell
Topic 2: coronavirus, sars, respiratory, nan, syndrome, mers-cov, severe, acute, sars-cov-2, sars-cov
Topic 3: cat, group, calve, sample, antibody, pig, study, day, infection, vaccine
Topic 4: virus, viral, sequence, rna, gene, strain, cell, genome, protein, replication
Topic 5: de, la, en, et, des, les, el, los, le, las
Topic 6: patient, respiratory, infection, virus, influenza, child, clinical, study, viral, pneumonia
Topic 7: disease, health, review, human, infectious, system, pathogen, research, new, development
Topic 8: case, health, covid-19, model, outbreak, study, disease, risk, transmission, epidemic
Topic 9: protein, cell, antibody, vaccine, bind, membrane, structure, peptide, domain, response


The dataset papers will be classified into the different previously modelled topics.

In [ ]:
docs_classified = lda.transform(vectorized_docs)
docs_classified[:5]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   51.3s remaining:   51.3s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:   54.7s finished


array([[8.19806364e-04, 8.19759651e-04, 8.19717485e-04, 8.19799204e-04,
        6.32149551e-01, 8.19736108e-04, 8.19731399e-04, 8.19853767e-04,
        8.19776380e-04, 3.61292269e-01],
       [1.25016209e-03, 1.25018439e-03, 1.25041330e-03, 1.25004015e-03,
        1.25041619e-03, 1.25002213e-03, 1.25008698e-03, 1.25015487e-03,
        1.25011639e-03, 9.88748403e-01],
       [1.09914209e-03, 5.70073444e-02, 8.97789669e-02, 1.09944172e-03,
        3.63453502e-01, 1.09893164e-03, 1.09949524e-03, 1.09914841e-03,
        1.09916923e-03, 4.83164858e-01],
       [8.92966492e-04, 3.25657831e-01, 8.93138852e-04, 8.92965763e-04,
        8.93204168e-04, 8.92900258e-04, 8.92976560e-04, 8.93080958e-04,
        8.93043218e-04, 6.67197893e-01],
       [8.40444342e-04, 8.40462004e-04, 8.40616402e-04, 7.62277498e-02,
        9.17048319e-01, 8.40364692e-04, 8.40446888e-04, 8.40494880e-04,
        8.40471210e-04, 8.40631060e-04]])

Finally, the top-5 PageRank-sorted papers belonging to each topic are displayed.

In [ ]:
docs_topics = docs_classified.argmax(1)
topic_papers = defaultdict(list)
all_papers = list(pageranks.keys())
for idx, topic_id in enumerate(docs_topics):
    topic_papers[topic_id].append(all_papers[idx])
    
for topic_id, papers in sorted(topic_papers.items(), key=lambda t: t[0]):
    print(f"Topic ID {topic_id}")
    sorted_papers = sorted(papers, reverse=True, key=lambda p: pageranks[p])
    for paper in sorted_papers[:5]:
        paper_as_markdown(paper)
    #print("\n", end="")

Topic ID 0



- **Title:** Mutations in the IGF-II pathway that confer resistance to lytic reovirus infection
- **Authors:** Sheng, Jinsong; Organ, Edward L; Hao, Chuanming; Wells, K Sam; Ruley, H Earl; Rubin, Donald H
- **Publish date/time:** 2004-08-27
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** BACKGROUND: Viruses are obligate intracellular parasites and rely upon the host cell for different steps in their life cycles. The characterization of cellular genes required for virus infection and/or cell killing will be essential for understanding viral life cycles, and may provide cellular targets for new antiviral therapies. RESULTS: A gene entrapment approach was used to identify candidate cellular genes that affect reovirus infection or virus induced cell lysis. Four of the 111 genes disrupted in clones selected for resistance to infection by reovirus type 1 involved the insulin growth factor-2 (IGF-II) pathway, including: the mannose-6-phosphate/IGF2 receptor (Igf2r), a protease associated with insulin growth factor binding protein 5 (Prss11), and the CTCF transcriptional regulator (Ctcf). The disruption of Ctcf, which encodes a repressor of Igf2, was associated with enhanced Igf2 gene expression. Plasmids expressing either the IGF-II pro-hormone or IGF-II without the carboxy terminal extension (E)-peptide sequence independently conferred high levels of cellular resistance to reovirus infection. Forced IGF-II expression results in a block in virus disassembly. In addition, Ctcf disruption and forced Igf2 expression both enabled cells to proliferate in soft agar, a phenotype associated with malignant growth in vivo. CONCLUSION: These results indicate that IGF-II, and by inference other components of the IGF-II signalling pathway, can confer resistance to lytic reovirus infection. This report represents the first use of gene entrapment to identify host factors affecting virus infection. Concomitant transformation observed in some virus resistant cells illustrates a potential mechanism of carcinogenesis associated with chronic virus infection.


- **Title:** A human in vitro model system for investigating genome-wide host responses to SARS coronavirus infection
- **Authors:** Ng, Lisa FP; Hibberd, Martin L; Ooi, Eng-Eong; Tang, Kin-Fai; Neo, Soek-Ying; Tan, Jenny; Krishna Murthy, Karuturi R; Vega, Vinsensius B; Chia, Jer-Ming; Liu, Edison T; Ren, Ee-Chee
- **Publish date/time:** 2004-09-09
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** BACKGROUND: The molecular basis of severe acute respiratory syndrome (SARS) coronavirus (CoV) induced pathology is still largely unclear. Many SARS patients suffer respiratory distress brought on by interstitial infiltration and frequently show peripheral blood lymphopenia and occasional leucopenia. One possible cause of this could be interstitial inflammation, following a localized host response. In this study, we therefore examine the immune response of SARS-CoV in human peripheral blood mononuclear cells (PBMCs) over the first 24 hours. METHODS: PBMCs from normal healthy donors were inoculated in vitro with SARS-CoV and the viral replication kinetics was studied by real-time quantitative assays. SARS-CoV specific gene expression changes were examined by high-density oligonucleotide array analysis. RESULTS: We observed that SARS-CoV was capable of infecting and replicating in PBMCs and the kinetics of viral replication was variable among the donors. SARS-CoV antibody binding assays indicated that SARS specific antibodies inhibited SARS-CoV viral replication. Array data showed monocyte-macrophage cell activation, coagulation pathway upregulation and cytokine production together with lung trafficking chemokines such as IL8 and IL17, possibly activated through the TLR9 signaling pathway; that mimicked clinical features of the disease. CONCLUSIONS: The identification of human blood mononuclear cells as a direct target of SARS-CoV in the model system described here provides a new insight into disease pathology and a tool for investigating the host response and mechanisms of pathogenesis.


- **Title:** The involvement of survival signaling pathways in rubella-virus induced apoptosis
- **Authors:** Cooray, Samantha; Jin, Li; Best, Jennifer M
- **Publish date/time:** 2005-01-04
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** Rubella virus (RV) causes severe congenital defects when acquired during the first trimester of pregnancy. RV cytopathic effect has been shown to be due to caspase-dependent apoptosis in a number of susceptible cell lines, and it has been suggested that this apoptotic induction could be a causal factor in the development of such defects. Often the outcome of apoptotic stimuli is dependent on apoptotic, proliferative and survival signaling mechanisms in the cell. Therefore we investigated the role of phosphoinositide 3-kinase (PI3K)-Akt survival signaling and Ras-Raf-MEK-ERK proliferative signaling during RV-induced apoptosis in RK13 cells. Increasing levels of phosphorylated ERK, Akt and GSK3β were detected from 24–96 hours post-infection, concomitant with RV-induced apoptotic signals. Inhibition of PI3K-Akt signaling reduced cell viability, and increased the speed and magnitude of RV-induced apoptosis, suggesting that this pathway contributes to cell survival during RV infection. In contrast, inhibition of the Ras-Raf-MEK-ERK pathway impaired RV replication and growth and reduced RV-induced apoptosis, suggesting that the normal cellular growth is required for efficient virus production.


- **Title:** Expression profile of immune response genes in patients with Severe Acute Respiratory Syndrome
- **Authors:** Reghunathan, Renji; Jayapal, Manikandan; Hsu, Li-Yang; Chng, Hiok-Hee; Tai, Dessmon; Leung, Bernard P; Melendez, Alirio J
- **Publish date/time:** 2005-01-18
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** BACKGROUND: Severe acute respiratory syndrome (SARS) emerged in later February 2003, as a new epidemic form of life-threatening infection caused by a novel coronavirus. However, the immune-pathogenesis of SARS is poorly understood. To understand the host response to this pathogen, we investigated the gene expression profiles of peripheral blood mononuclear cells (PBMCs) derived from SARS patients, and compared with healthy controls. RESULTS: The number of differentially expressed genes was found to be 186 under stringent filtering criteria of microarray data analysis. Several genes were highly up-regulated in patients with SARS, such as, the genes coding for Lactoferrin, S100A9 and Lipocalin 2. The real-time PCR method verified the results of the gene array analysis and showed that those genes that were up-regulated as determined by microarray analysis were also found to be comparatively up-regulated by real-time PCR analysis. CONCLUSIONS: This differential gene expression profiling of PBMCs from patients with SARS strongly suggests that the response of SARS affected patients seems to be mainly an innate inflammatory response, rather than a specific immune response against a viral infection, as we observed a complete lack of cytokine genes usually triggered during a viral infection. Our study shows for the first time how the immune system responds to the SARS infection, and opens new possibilities for designing new diagnostics and treatments for this new life-threatening disease.

...


- **Title:** Crystal structure of murine sCEACAM1a[1,4]: a coronavirus receptor in the CEA family
- **Authors:** Tan, Kemin; Zelus, Bruce D.; Meijers, Rob; Liu, Jin-huan; Bergelson, Jeffrey M.; Duke, Norma; Zhang, Rongguang; Joachimiak, Andrzej; Holmes, Kathryn V.; Wang, Jia-huai
- **Publish date/time:** 2002-05-01
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** CEACAM1 is a member of the carcinoembryonic antigen (CEA) family. Isoforms of murine CEACAM1 serve as receptors for mouse hepatitis virus (MHV), a murine coronavirus. Here we report the crystal structure of soluble murine sCEACAM1a[1,4], which is composed of two Ig-like domains and has MHV neutralizing activity. Its N-terminal domain has a uniquely folded CC′ loop that encompasses key virus-binding residues. This is the first atomic structure of any member of the CEA family, and provides a prototypic architecture for functional exploration of CEA family members. We discuss the structural basis of virus receptor activities of murine CEACAM1 proteins, binding of Neisseria to human CEACAM1, and other homophilic and heterophilic interactions of CEA family members.


- **Title:** Synthesis of a novel hepatitis C virus protein by ribosomal frameshift
- **Authors:** Xu, Zhenming; Choi, Jinah; Yen, T.S.Benedict; Lu, Wen; Strohecker, Anne; Govindarajan, Sugantha; Chien, David; Selby, Mark J.; Ou, Jing‐hsiung
- **Publish date/time:** 2001-07-16
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** Hepatitis C virus (HCV) is an important human pathogen that affects ∼100 million people worldwide. Its RNA genome codes for a polyprotein, which is cleaved by viral and cellular proteases to produce at least 10 mature viral protein products. We report here the discovery of a novel HCV protein synthesized by ribosomal frameshift. This protein, which we named the F protein, is synthesized from the initiation codon of the polyprotein sequence followed by ribosomal frameshift into the −2/+1 reading frame. This ribosomal frameshift requires only codons 8–14 of the core protein‐coding sequence, and the shift junction is located at or near codon 11. An F protein analog synthesized in vitro reacted with the sera of HCV patients but not with the sera of hepatitis B patients, indicating the expression of the F protein during natural HCV infection. This unexpected finding may open new avenues for the development of anti‐HCV drugs.


- **Title:** Structure of coronavirus main proteinase reveals combination of a chymotrypsin fold with an extra α-helical domain
- **Authors:** Anand, Kanchan; Palm, Gottfried J.; Mesters, Jeroen R.; Siddell, Stuart G.; Ziebuhr, John; Hilgenfeld, Rolf
- **Publish date/time:** 2002-07-01
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** The key enzyme in coronavirus polyprotein processing is the viral main proteinase, M(pro), a protein with extremely low sequence similarity to other viral and cellular proteinases. Here, the crystal structure of the 33.1 kDa transmissible gastroenteritis (corona)virus M(pro) is reported. The structure was refined to 1.96 Å resolution and revealed three dimers in the asymmetric unit. The mutual arrangement of the protomers in each of the dimers suggests that M(pro) self-processing occurs in trans. The active site, comprised of Cys144 and His41, is part of a chymotrypsin-like fold that is connected by a 16 residue loop to an extra domain featuring a novel α-helical fold. Molecular modelling and mutagenesis data implicate the loop in substrate binding and elucidate S1 and S2 subsites suitable to accommodate the side chains of the P1 glutamine and P2 leucine residues of M(pro) substrates. Interactions involving the N-terminus and the α-helical domain stabilize the loop in the orientation required for trans-cleavage activity. The study illustrates that RNA viruses have evolved unprecedented variations of the classical chymotrypsin fold.


- **Title:** Prokaryotic-style frameshifting in a plant translation system: conservation of an unusual single-tRNA slippage event
- **Authors:** Napthine, Sawsan; Vidakovic, Marijana; Girnary, Roseanne; Namy, Olivier; Brierley, Ian
- **Publish date/time:** 2003-08-01
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** Ribosomal frameshifting signals are found in mobile genetic elements, viruses and cellular genes of prokaryotes and eukaryotes. Typically they comprise a slippery sequence, X XXY YYZ, where the frameshift occurs, and a stimulatory mRNA element. Here we studied the influence of host translational environment and the identity of slippery sequence-decoding tRNAs on the frameshift mechanism. By expressing candidate signals in Escherichia coli, and in wheatgerm extracts depleted of endogenous tRNAs and supplemented with prokaryotic or eukaryotic tRNA populations, we show that when decoding AAG in the ribosomal A-site, E.coli tRNA(Lys) promotes a highly unusual single-tRNA slippage event in both prokaryotic and eukaryotic ribosomes. This event does not appear to require slippage of the adjacent P-site tRNA, although its identity is influential. Conversely, asparaginyl-tRNA promoted a dual slippage event in either system. Thus, the tRNAs themselves are the main determinants in the selection of single- or dual-tRNA slippage mechanisms. We also show for the first time that prokaryotic tRNA(Asn) is not inherently ‘unslippery’ and induces efficient frameshifting when in the context of a eukaryotic translation system.


- **Title:** SseG, a virulence protein that targets Salmonella to the Golgi network
- **Authors:** Salcedo, Suzana P.; Holden, David W.
- **Publish date/time:** 2003-10-01
- **Linked references:** 0
- **Linked referenced by:** 0
- **Abstract:** Intracellular replication of the bacterial pathogen Salmonella enterica occurs in membrane-bound compartments called Salmonella-containing vacuoles (SCVs). Maturation of the SCV has been shown to occur by selective interactions with the endocytic pathway. We show here that after invasion of epithelial cells and migration to a perinuclear location, the majority of SCVs become surrounded by membranes of the Golgi network. This process is dependent on the Salmonella pathogenicity island 2 type III secretion system effector SseG. In infected cells, SseG was associated with the SCV and peripheral punctate structures. Only bacterial cells closely associated with the Golgi network were able to multiply; furthermore, mutation of sseG or disruption of the Golgi network inhibited intracellular bacterial growth. When expressed in epithelial cells, SseG co-localized extensively with markers of the trans-Golgi network. We identify a Golgi-targeting domain within SseG, and other regions of the protein that are required for localization of bacteria to the Golgi network. Therefore, replication of Salmonella in epithelial cells is dependent on simultaneous and selective interactions with both endocytic and secretory pathways.

---

In [ ]:
# tell nbdev to generate library from notebooks
from nbdev.export import *
notebook2script()

Converted 00_downloader.ipynb.
Converted 01_references.ipynb.
Converted 02_representations_and_lda.ipynb.
Converted 03_hierarchical_topic_modelling.ipynb.
Converted 98_risotto_precook.ipynb.
Converted 99_risotto_gui.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.


In [ ]:
# this code is here for cosmetic reasons
from IPython.core.display import HTML
from urllib.request import urlopen
HTML(urlopen('https://raw.githubusercontent.com/lmarti/jupyter_custom/master/custom.include').read().decode('utf-8'))

---